In [1]:
from CNN.CNN import MultivariateCNN
from CNN.cnn_utils import train_cnn, get_train_data
from RF_XGB.RandomForest import train_random_forest
from models_utils.GLOBALS import *
from models_utils.utils import get_results
import pandas as pd

In [ ]:
# get train data
train_data = pd.read_csv('train.csv', index_col=0)
train_data['activity'] = train_data['activity'].map(activity_id_mapping)
data_type_1 = train_data[train_data['sensor'] == 'vicon'].reset_index()
data_type_2 = train_data[train_data['sensor'] == 'smartwatch'].reset_index()

In [ ]:
# sizes of padding/cutting
target_size_type1 = 3000
target_size_type2 = 1169

In [ ]:
# train or load models
train_or_load_autoencoders = 'load'
if train_or_load_autoencoders == 'train':
    model_CNN_type1 = train_cnn('Type1CNNRFModel', data_type_1, '1', target_size_type1, 1, 64, 0.001)
    model_CNN_type2 = train_cnn('Type2CNNRFModel', data_type_2, '2', target_size_type2, 1, 64, 0.001)
elif train_or_load_autoencoders == 'load':
    model_CNN_type1 = MultivariateCNN(3, target_size_type1, 18).to(device)
    model_CNN_type1.load_state_dict(torch.load('Type1CNNRFModel.pth'))
    model_CNN_type2 = MultivariateCNN(3, target_size_type2, 18).to(device)
    model_CNN_type2.load_state_dict(torch.load(f'Type2CNNRFModel.pth'))
else:
    raise ValueError('Wrong train or load')

In [ ]:
# calculate or load train data
embedding_names = [f'embedding_feature_{i + 1}' for i in range(18)]
calculate_or_load_train_data = 'calculate'
if calculate_or_load_train_data == 'calculate':
    data_type_1, data_type_2 = get_train_data('train_CNN_data.csv', model_CNN_type1, model_CNN_type2,
                                              target_size_type1, target_size_type2, is_autoencoder=False)
elif calculate_or_load_train_data == 'load':
    data_type_1 = pd.read_csv('train_CNN_data_type1.csv')
    data_type_2 = pd.read_csv('train_CNN_data_type2.csv')

In [ ]:
# train rf
cols_to_drop_type_1 = ['activity', 'dominant_freq_y', 'count_x', 'count_y', 'count_z', 'dominant_freq_x']
rf_classifier_type1, label_encoder_type1 = train_random_forest(data_type_1, cols_to_drop_type_1, n_estimators=200)
cols_to_drop_type_2 = ['activity', 'dominant_freq_y', 'count_x', 'count_y', 'count_z', 'dominant_freq_x']
rf_classifier_type2, label_encoder_type2 = train_random_forest(data_type_2, cols_to_drop_type_2, n_estimators=200)

In [43]:
# get and save results for test data
results = get_results(model_CNN_type1, model_CNN_type2, rf_classifier_type1, rf_classifier_type2,
                      label_encoder_type1, label_encoder_type2, target_size_type1, target_size_type2,
                      cols_to_drop_type_1, cols_to_drop_type_2, embedding_names)
results.to_csv('results_CNN_RF.csv', index=False)